In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
 def ivt_NH(df_int,Fs,k):
    
    #Loop para encontrar el umbral de velocidad. A la salida tiene los picos detectados de las supuestas sacadas
    fact = np.array([6.38,4.76,3.55,2.65,1.97,1.47,1.1,0.82,0.61,0.46,0.34,0.25,0.19,0.14,0.11])
    num1=0
    for n in range(0,fact.size):
        (w_peaks1,w_th,mu1,sigma1)=threshold_saccade(df_int,fact[n],Fs) #Itera el primer umbral con fact[n]
        if w_peaks1.size>20: #Para que 'arranque' que siga iterando hasta que encuentre al menos 20 sacadas
            if n>0:
                if w_peaks1.size>0 and num1!=0:
                    if (100*(w_peaks1.size-num1)/num1)<= 10: #Itera en fact[n] hasta que el número de picos que encuentra en la 
                        break                                #nueva iteración es menor al 10% que en la anterior
                else:
                    continue
        num1=w_peaks1.size
    print('\033[91m' + 'Umbral velocidad:' + '\033[0m', w_th)

    #Detecta indice del onset
    (index_onset,w_peaks1)=onset_saccade(df_int,w_peaks1,mu1,sigma1,fact[n])
    w_peaks1_sz=w_peaks1.size

    #Detecta indice offset
    (index_offset,index_onset,w_peaks1,w_th_offset)=offset_saccade(df_int,w_peaks1,index_onset,mu1,sigma1,fact[n],Fs)
    w_peaks1_sz=w_peaks1.size
    
    #Pasa detección de sacadas y datos a frame.
    (index_onset,index_offset,data_frame)=data_to_frame(df_int,index_onset,index_offset)
    saccades_frame=saccades_to_frame(df_int,index_onset,index_offset)
    fix_frame=fix_to_frame(df_int,saccades_frame)
    print('\033[91m' + 'Cantidad de sacadas en pos-procesamiento:' + '\033[0m',saccades_frame["onset"].size)    
        
    return (saccades_frame,fix_frame,data_frame,w_th)

In [ ]:
#Loop para hallar el umbral de velocidad y los picos que corresponden a las sacadas.
#Al final los indices de los picos mayores que el umbral quedan guardados en w_peaks1, el umbral en w_th
def threshold_saccade(df_int,fact,Fs):
    #Usa el módulo de velocidad
    w_interp=df_int["vpr"]

    #Busca los picos en la velocidad
    w_peaks, _ = find_peaks(w_interp)
    
    #Primer umbral para la velocidad y picos. Elijo como primer umbral (max-min)/4
    w_th=(np.amax(w_interp[w_peaks])-np.amin(w_interp[w_peaks]))/4
    w_th1=0
    i=0
    if Fs == 1000:
        imax=100 #Es la cantidad de iteraciones que va a hacer hasta encontrar el umbral
        dist=10 #Distancia mínima permitida entre picos (cantidad de puntos)
        lim=1.0 #Tolerancia entre iteraciones del umbral para corte
    if Fs == 90 or Fs==60:
        imax=100
        dist=4
        lim=1.0
            
    while i<imax:
        w_peaks1,_= find_peaks(w_interp,height=w_th,distance=dist) #Busca los picos mayores que el umbral
        index_peaks1=w_peaks[~np.isin(w_peaks,w_peaks1)] #busca los indices de esos picos para sacarlos
        #Busco mu=promedio y sigma=desvio standard del resto de los picos
        mu1=np.nanmean(w_interp[index_peaks1])
        sigma1=np.nanstd(w_interp[index_peaks1])
        w_th1=mu1+fact*sigma1 #Nuevo umbral. Cambie a w_th1=mu1+fact*sigma1 en vez de 6*sigma1 como dice el paper
        if np.abs(w_th1-w_th) < lim: #Si la diferencia entre umbrales es menor que lim, corta y se queda con el último
            break
        else:
            w_th=w_th1 #Sino sigue iterando hasta llegar a imax
            i += 1
            continue
        
    return (w_peaks1,w_th,mu1,sigma1)

In [ ]:
#Onset de sacadas. Uso mu1 y sigma1 del bloque anterior, el promedio y desvio de los picos menores que el umbral.
#Los indices del onset quedan guardados en index_onset
def onset_saccade(df_int,w_peaks1,mu1,sigma1,val):
    #Módulo de la velocidad
    w_interp=df_int["vpr"]

    w_thz=mu1+val*sigma1
    w_peaks1_sz=w_peaks1.size
    index_onset = np.array([],dtype=int)
    indx_delete = np.array([],dtype=int)
    i=0
    imax=100
    for n in range(0,w_peaks1_sz,1):
        for i in range(1,imax,1): #Busca en i desde el pico del umbral hacia atrás.
            if w_peaks1[n]-i < 0:
                index_onset = np.append(index_onset,-1) #Esto es si se pasa del principio de los índices
                break
            if np.isnan(w_interp[w_peaks1[n]-i]): #Chequea si en la busqueda hay Nan, guarda el indice para borrar
                indx_delete=np.append(indx_delete,n)
                break
            if w_interp[w_peaks1[n]-i] < w_thz: #Si el índice está por debajo del umbral
                if Fs==1000: #Si la frecuencia es 1000 Hx, por la densidad de puntos, busca una inflexión.
                    if w_interp[w_peaks1[n]-i]-w_interp[w_peaks1[n]-i+1] >= 0:
                        index_onset = np.append(index_onset,w_peaks1[n]-i)
                        break
                else:
                    index_onset = np.append(index_onset,w_peaks1[n]-i) #Si la frecuencia es 90 Hz, se queda con el primer punto debajo del umbral
                    break
            if i==imax-1:
                indx_delete=np.append(indx_delete,n) #Si llegó al valor imax y no encontró un punto debajo del umbral, borra ese pico
                break
        else:
            continue
            
#Borra los picos cerca de Nan y recalcula el tamaño y donde no encontró el onset
    w_peaks1=np.delete(w_peaks1,indx_delete)
    return (index_onset,w_peaks1)

In [ ]:
#Offset de sacadas. Uso w_thz del bloque anterior.
#Los indices del offset quedan guardados en index_offset
def offset_saccade(df_int,w_peaks1,index_onset,mu1,sigma1,val,Fs):
    w_interp=df_int["vpr"]

    index_offset = np.array([],dtype=int)
    w_th_offset = np.array([],dtype=float)
    noise_index=np.int_(np.ceil(Fs*40/1000)) #Indice de 'ruido'. Ventana de puntos donde mide el ruido de la señal para tener en cuenta las PSO
    indx_delete = np.array([],dtype=int)
    w_thz=mu1+val*sigma1
    w_peaks1_sz=w_peaks1.size
    theta_sz=w_interp.size
    i=0
    imax=100
    for n in range(0,w_peaks1_sz,1):
    #Calcula el factor de ruido local, en una ventana de 40 ms antes del comienzo de la sacada
        if index_onset[n]-noise_index < 0:
                mut=np.nanmean(w_interp[0:index_onset[n]])
                sigmat=np.nanstd(w_interp[0:index_onset[n]])
        else:
            mut=np.nanmean(w_interp[index_onset[n]-noise_index:index_onset[n]])
            sigmat=np.nanstd(w_interp[index_onset[n]-noise_index:index_onset[n]])
        w_tht=mut+3*sigmat
        w_th_offset=np.append(w_th_offset,0.7*w_thz+0.3*w_tht) #umbral offset
        #Busca 'hacia adelante' del pico
        for i in range(1,imax,1):
            if i==imax-1:
                indx_delete=np.append(indx_delete,n) #Si llegó a imax y no encontró offset borra el pico
                break
            if w_peaks1[n]+i >= theta_sz-1: #Si llegó al final de la lista borra el pico
                index_offset = np.append(index_offset,-1)
                break
            if np.isnan(w_interp[w_peaks1[n]+i]):
                indx_delete=np.append(indx_delete,n) #Guarda los indices donde hay Nan alrededor para borrar despues
                break
            if w_interp[w_peaks1[n]+i] < w_th_offset[n]: #Si la velocidad llegó a ser menor que el umbral nuevo
                if Fs==1000: #Para 1000 Hz busca inflexión
                    if w_interp[w_peaks1[n]+i]-w_interp[w_peaks1[n]+i+1] <= 0:
                        index_offset = np.append(index_offset,w_peaks1[n]+i+1)
                        break
                else: #Para 90 Hz se queda con ese punto
                    index_offset = np.append(index_offset,w_peaks1[n]+i)
                    break
        else:
            continue
            
#Borra donde se encontro Nan y recalcula el tamaño
    w_peaks1=np.delete(w_peaks1,indx_delete)
    index_onset=np.delete(index_onset,indx_delete)
    return (index_offset,index_onset,w_peaks1,w_th_offset)